In [65]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [66]:
# 1. Load the similar playlist id
df_pid = pd.read_csv("similar_pids.csv")
df_nf = pd.read_csv("new_features.csv")
pid_list = df_pid.iloc[:, 0]
print(pid_list)

0       5
1      31
2      36
3      38
4      40
5      68
6      95
7     105
8     107
9     123
10    124
11    136
12    170
13    173
Name: 1, dtype: int64


In [67]:
filtered_songs = df_nf[df_nf["pid"].isin(pid_list)].drop(columns=['pid'], errors='ignore').drop_duplicates().reset_index(drop=True)
# print(filtered_songs)
print(filtered_songs.shape)

(174, 28)


In [68]:
filtered_songs_numbers_only = filtered_songs.drop(columns=['track_id', 'track_uri', 'track_name', 'artist_name', 'id', 'name', 'album', 'album_id', 'artists', 'artist_ids', 'track_number', 'disc_number', 'explicit', 'duration_ms', 'time_signature', 'year', 'release_date'], errors='ignore')
# print(filtered_songs_numbers_only)
#filtered_songs_numbers_only_dict = {filtered_songs.loc[i, 'track_id']: filtered_songs_numbers_only[i] for i in range(len(filtered_songs))}
filtered_songs_numbers_only_dict = {
    filtered_songs.loc[i, 'track_id']: filtered_songs_numbers_only.iloc[i].to_dict()
    for i in range(len(filtered_songs))
}
# print(filtered_songs_numbers_only_dict)

In [69]:
# 2. Train K Nearest Neighbors model
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')
all_track_ids_list = list(filtered_songs_numbers_only_dict.keys())
all_vectors = np.array([list(v.values()) for v in filtered_songs_numbers_only_dict.values()], dtype=float)
# nbrs.fit(all_vectors, all_track_ids_list)
nbrs.fit(all_vectors)

NearestNeighbors(algorithm='ball_tree', n_neighbors=10)

In [70]:
def recommend_nearest(target_track_id, top_n=10):
    if target_track_id not in all_track_ids_list:
        print(f"Track {target_track_id} not found.")
        return []
    target_vector = np.array(list(filtered_songs_numbers_only_dict[target_track_id].values())).reshape(1, -1)
    distances, indices = nbrs.kneighbors(target_vector)
    recommendations = [(all_track_ids_list[idx], distances[0][j]) for j, idx in enumerate(indices[0])]
    return recommendations[:top_n][0]

In [75]:
recommendation_songs_rankings = []
for i in range(len(all_track_ids_list)):
    recommendation_songs_rankings.append(recommend_nearest(all_track_ids_list[i], 1))
print(recommendation_songs_rankings)

[('0CAfXk7DXMnon4gLudAp7J', 0.0), ('5uuJruktM9fMdN9Va0DUMl', 0.0), ('6e8Ou0wiqAzIpWb2eSxll8', 0.0), ('27AHAtAirQapVldIm4c9ZX', 0.0), ('6YUTL4dYpB9xZO5qExPf05', 0.0), ('3DmW6y7wTEYHJZlLo1r6XJ', 0.0), ('3cHyrEgdyYRjgJKSOiOtcS', 0.0), ('7MmG8p0F9N3C4AXdK6o6Eb', 0.0), ('1rIKgCH4H52lrvDcz50hS8', 0.0), ('4viUsO0ZnlZrVCM945K6on', 0.0), ('3NLnwwAQbbFKcEcV8hDItk', 0.0), ('29gRHViWSpSV7UWb5e3QCv', 0.0), ('4tCtwWceOPWzenK2HAIJSb', 0.0), ('6M6Tk58pQvABy6ru66dY3d', 0.0), ('59A6lBd9QmcD8rQTZPj1xD', 0.0), ('5j4mKTvOu0cr4cooq57sm3', 0.0), ('1l7n3I8SoPR4OyLnTm1uqK', 0.0), ('6ukvsBzq4d1vBsAUmz7ZVt', 0.0), ('0azC730Exh71aQlOt9Zj3y', 0.0), ('3lJA6dGqvaKtVxqHbTtQiQ', 0.0), ('5H7CwzYZ60e7w69tX4ivQN', 0.0), ('2aW9GjctURzw8Yt7BC3wJY', 0.0), ('4c9WmjVlQMr0s1IjbYO52Z', 0.0), ('7uKcScNXuO3MWw6LowBjW1', 0.0), ('7wZUrN8oemZfsEd1CGkbXE', 0.0), ('6I9VzXrHxO9rA9A5euc8Ak', 0.0), ('2aBxt229cbLDOvtL7Xbb9x', 0.0), ('5VGlqQANWDKJFl0MBG3sg2', 0.0), ('1Oi2zpmL81Q0yScF1zxaC0', 0.0), ('1AWQoqb9bSvzTjaLralEkT', 0.0), ('6LGwYMX

In [76]:
# Example Usage
recommendation_songs_rankings = [
    recommend_nearest(track_id, 1)[0]  # Extract first element
    for track_id in df_nf[df_nf["pid"] == 0]["track_id"]
]
print(np.array(recommendation_songs_rankings).shape)
print(recommendation_songs_rankings)

Track 7H6ev70Weq6DdpZyyTmUXk not found.


IndexError: list index out of range